## Utility Price 2015-2018: $p^b$
Based on [EDF](https://www.edf.fr/)'s state regulated electricity tariffs (aka "Tarif Bleu") for a subscribed apparent power of 9kVA. The regulator publishes the electricity prices here:
- [01/11/2014 - 31/07/2015](https://www.legifrance.gouv.fr/jo_pdf.do?id=JORFTEXT000033172637)
- [01/08/2015 - 31/07/2016](https://www.legifrance.gouv.fr/jo_pdf.do?id=JORFTEXT000030954456)
- [01/08/2016 - 31/07/2017](https://www.edf.fr/sites/default/files/contrib/collectivite/electricite-et-gaz/CGV%2018avril/jo_du_29_juillet_2016_trv.pdf)
- [01/08/2017 - 31/01/2018](https://www.legifrance.gouv.fr/jo_pdf.do?id=JORFTEXT000035297675)
- [01/02/2018 - 31/07/2018](https://www.legifrance.gouv.fr/jo_pdf.do?id=JORFTEXT000036559814)
- [01/08/2018 - onwards.](https://www.legifrance.gouv.fr/jo_pdf.do?id=JORFTEXT000037262170)

These prices exclude taxes and transportation fees. The [current prices including these taxes and fees](https://particulier.edf.fr/content/dam/2-Actifs/Documents/Offres/Grille_prix_Tarif_Bleu.pdf) are published by EDF directly. The "Option Base" offers a flat price throughout the day. The "Option Heures Creuses" has a higher price for peak (6:00-22:00) than off-peak (22:00-6:00) hours. EDF refers to the peak hours as "Heures Pleines (HP)" and to the off-peak hours as "Heures Creuses (HC)".  The prices of both of these options do not usually change from one day to the next. They only change, when EDF changes its electricity tariffs, which is one to two times per year.

Conversely, the "Option Tempo" is a pricing scheme in which each day is either a high-price ("Rouge"), medium-price ("Blanc") or low-price ("Bleu") day. The price level of each day is announced before noon the preceeding day. This means that it is known when the battery owner decides on her market actions $x^b,x^r$. RTE keeps track of the [daily price levels](https://www.services-rte.com/fr/visualisez-les-donnees-publiees-par-rte/calendrier-des-offres-de-fourniture-de-type-tempo.html) and offers an [API](https://data.rte-france.com/catalog/-/api/user_guide/236629) to download them. Alternatively, the data can also be downloaded using RTE's [eco2mix](https://www.rte-france.com/fr/eco2mix/eco2mix-telechargement) platform.

In [1]:
# import libraries
import pandas as pd
import numpy as np
from datetime import datetime

We fill a dataframe with the prices found in government publications. These prices do not include taxes and transportation fees.

In [2]:
df_no_tax = pd.DataFrame({'Flat': [0.0947, 0.0932, 0.0898, 0.0915, 0.0915, 0.0902],\
                         'HP': [0.107, 0.1043, 0.0979, 0.1007, 0.1007, 0.0995], \
                         'HC': [0.0654, 0.0638, 0.0738, 0.0723, 0.0716, 0.0703], \
                         'Bleu HP': [0.0618, 0.0635, 0.0744, 0.0788, 0.0785, 0.0787], \
                         'Blanc HP': [0.0973, 0.1, 0.1074, 0.1042, 0.1037, 0.0978], \
                         'Rouge HP': [0.4721, 0.4852, 0.4026, 0.4261, 0.4248, 0.4190], \
                         'Bleu HC' : [0.0472, 0.0485, 0.0575, 0.0609, 0.0607, 0.0599], \
                         'Blanc HC': [0.0771, 0.0793, 0.0846, 0.0805, 0.0802, 0.0725], \
                         'Rouge HC': [0.1636, 0.1681, 0.1480, 0.1140, 0.1135, 0.0782]}, \
                         index = [datetime(2015,1,1,0,0), datetime(2015,8,1,0,0), datetime(2016,8,1,0,0), datetime(2017,8,1,0,0), datetime(2018,2,1,0,0), datetime(2018,8,1,0,0)])

For comparison, we fill a dataframe with the 2018 prices including taxes and transportation fees. Our aim is to estimate these prices for 2015-17.

In [3]:
df_with_tax = pd.DataFrame({'Flat': [0.1483, 0.1450],\
                            'HP': [0.1593, 0.1579],\
                            'HC': [0.1244, 0.1228],\
                            'Bleu HP': [0.1327, 0.1329],\
                            'Blanc HP': [0.1629, 0.1558],\
                            'Rouge HP': [0.5482, 0.5413],\
                            'Bleu HC': [0.1113, 0.1104],\
                            'Blanc HC': [0.1347, 0.1255],\
                            'Rouge HC': [0.1747, 0.1323]},\
                            index = [datetime(2018,2,1,0,0), datetime(2018,8,1,0,0)])

Estimate the average ratio between costs before and after tax.

In [4]:
df_ratio = df_with_tax.div(df_no_tax.loc[df_with_tax.index.values])
df_ratio

,Flat,HP,HC,Bleu HP,Blanc HP,Rouge HP,Bleu HC,Blanc HC,Rouge HC
2018-02-01,1.620765,1.581927,1.737430,1.690446,1.570878,1.290490,1.833608,1.679551,1.539207
2018-08-01,1.607539,1.586935,1.746799,1.688691,1.593047,1.291885,1.843072,1.731034,1.691816


In [5]:
# Estimate the prices with tax based on this ratio
df_est_tax = pd.concat((df_no_tax.iloc[:4,:]*df_ratio.mean(),df_with_tax))

Expand the prices to a 30 min time resolution

In [6]:
df = df_est_tax.reindex(pd.date_range('01-01-2015 00:00:00', '12-31-2018 23:30:00', freq='30min'), method='ffill')

Add day dependent pricing

In [7]:
df_tempo = pd.concat([pd.read_excel('Tempo_2014-2015.xlsx'), pd.read_excel('Tempo_2015-2016.xlsx'), pd.read_excel('Tempo_2016-2017.xlsx'), pd.read_excel('Tempo_2017-2018.xlsx'), pd.read_excel('Tempo_2018-2019.xlsx')])
df_tempo.dropna(inplace=True)
df_tempo.set_index('Date', inplace=True)
df_tempo.columns = ['Color']
df_tempo = df_tempo.reindex(pd.date_range('01-01-2015 00:00:00', '12-31-2018 23:30:00', freq='30min'), method='ffill')
df = pd.concat([df, df_tempo], axis = 1)

In [8]:
# Apply on- and off-peak hours as well as the daily price regime
pb = []
pr = []
for k in range(len(df.index)):
        if df.index[k].hour in range(6,22):
            pb.append(df['HP'].iloc[k])
            if df.Color[k] == 'BLANC':
                pr.append(df['Blanc HP'].iloc[k])
            elif df.Color[k] == 'BLEU':
                pr.append(df['Bleu HP'].iloc[k])
            else:
                pr.append(df['Rouge HP'].iloc[k])
        else:
            pb.append(df['HC'].iloc[k])
            if df.Color[k] == 'BLANC':
                pr.append(df['Blanc HC'].iloc[k])
            elif df.Color[k] == 'BLEU':
                pr.append(df['Bleu HC'].iloc[k])
            else:
                pr.append(df['Rouge HC'].iloc[k])
df['HPHC'] = pb
df['Tempo'] = pr

Plot the utility price

In [ ]:
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot({
    "data": [go.Scatter(x=df.index, y=df['Flat'], name='Flat'),
             go.Scatter(x=df.index, y=df['HPHC'], name='HPHC'),
             go.Scatter(x=df.index, y=df['Tempo'], name='Tempo')],
    "layout": go.Layout(yaxis=dict(title="Utility Price (€/kW/h)"))
})

Save relevant data

In [14]:
df[['Flat','HPHC','Tempo']].to_hdf('pb.h5', key='df')